# 1. Работаем с выгрзукой данных из базы

In [1]:
# Импортиурем библиотеку для работы с БД на python
import psycopg2

# Пытаемся подключиться к БД, при ошибке выводим предупреждение
try: 
    conn = psycopg2.connect(dbname='postgres', user='postgres', password='123', host='localhost')
    # Получаем объект курсора, для запросов к БД
    cursor = conn.cursor()
except:
    print('Неудалось подключиться к БД!')

In [2]:
# Извлекаем данные запросом из БД
cursor.execute('SELECT name, description, sallary, region_name FROM jobschema.joblist')
# Загружаем данные из запроса и преобрабатываем
all_info = list(map(list, cursor.fetchall()))

In [3]:
# Извлекаем данные запросом из БД
cursor.execute('SELECT skills FROM jobschema.joblist')
# Загружаем данные из запроса и предобрабатываем
all_skills = list(set(sum([i[0].split(';') for i in list(map(list, cursor.fetchall()))], [])))
# Удаляем пустой элемент из списка
all_skills.pop(0)
# Выходим из курсора
cursor.close()
# Отключаемся от БД
conn.close()

# 2. Предобрабатываем данные

In [4]:
# Импортиурем библиотеку для леметизации предложений
import spacy
# Импортируем бибилиотеку для работы со строками
import re

# Загружаем преобученную модель для лемитизации русских слов
nlp = spacy.load("ru_core_news_sm")

# Циклом проходимся по всем описаниям из выгруженных данных
for i in range(len(all_info)):
    # Удаляем из описания знаки препинания 
    description = re.sub(r'[.,?:!"-()\';{}]', r'', all_info[i][1])
    # Иницилизируем объект nlp
    doc = nlp(description)
    # Заменяем прошлое описание из выгруженных данных на леметизированное описание
    all_info[i][1] = " ".join([token.lemma_ for token in doc])


# 3. Реализация NLP работы с данными

In [5]:
# Импортиурем бибилиотеку для работы с нейросетями
import torch
# Импортиурем библиотеку для загрузки оттуда предобученного трасформера типа BERT
from transformers import BertTokenizer, BertModel
# Импортиурем из библиотеки реализацию алгоритма к-ближайших соседей
from sklearn.neighbors import KNeighborsClassifier

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
# Загрузка модели RuBERT
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")

# Создание размеченного словаря, с помщью которого мы обучим нашу модель
d = {i: "Навык" for i in all_skills} | {i: "Не навык" for i in ["яблоко", "пенек", "машина", "дерево", "карандаш", "стул", "стол", "pencil", "pen", "car", "door", "tree", "window", "окно"]}

# Передаем X данные для обучения
words = list(d.keys())

# Массив для получение векторных представлений слов с помощью RuBERT
word_embeddings = []
# Циклом проходимся по всем переданным данным
for word in words:
    # Токенизация слов и преобразование в тензор
    inputs = tokenizer(word, return_tensors="pt", max_length=128, truncation=True)
    with torch.no_grad():
        # Получение выходного состояния модели BERT
        outputs = model(**inputs)
        # Усреднение по токенам
        word_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        # Добавляем эмбеддинг в ранее созданый массив
        word_embeddings.append(word_embedding)

# Передаем в переменную импортированный ранее класс для реализации алгоритма K-NN
knn_classifier = KNeighborsClassifier(n_neighbors=1) 
# Фитим нашу модель
knn_classifier.fit(word_embeddings, list(d.values()))

# Предиктим для проверки наши данные
predicted_classes = knn_classifier.predict(word_embeddings)

count = 0
# Вывод результатов примера классификации слов
for word, predicted_class in zip(words, predicted_classes):
    print(word, "->", predicted_class)
    if count == 5:
        break
    count += 1

c:\Users\pupki\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of 

uml -> Навык
microsoft powerpoint -> Навык
многозадачность -> Навык
самоорганизованность -> Навык
teradata -> Навык
моделирование рисков -> Навык


In [6]:
# Импортиурем бибилиотеку для работы с нейросетями
import torch
# Импортиурем библиотеку для загрузки оттуда предобученного трасформера типа BERT
from transformers import BertTokenizer, BertModel

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
# Загрузка модели RuBERT
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")

# Вытаскивание описания из данных
texts = [info[1] for info in all_info]

# Создание списка для хранения предсказанных классов для каждого предложения
predicted_classes_per_sentence = []

# Циклом проходимся по всем переданным данным
for text in texts:
    # Из предложения делаем массив
    words = text.split()
    # Создаем массив для эмбеддинга слов
    word_embeddings = []
    # Проходимся по всем словам из предложения
    for word in words:
        # Токенизация слов и преобразование в тензор
        inputs = tokenizer(word, return_tensors="pt", max_length=128, truncation=True)
        with torch.no_grad():
            # Получение выходного состояния модели BERT
            outputs = model(**inputs)
            # Усреднение по токенам
            word_embedding = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
            # Добавляем полученный эмбеддинг в ранее созданный массив
            word_embeddings.append(word_embedding)
    
    # Классификация слов текущего предложения
    predicted_classes = knn_classifier.predict(word_embeddings)
    # Добавляем в ранее созданный массив предсказанные классы
    predicted_classes_per_sentence.append(predicted_classes)


# Добавляем в каждый массив с данными еще одно поле, где будем хранить навыки из описания
for i in range(len(all_info)):
    all_info[i].append([])

# Счетчик для перехода между данными вакансий
count = 0
# Проходимся по предложениям и их классам
for sentence, predicted_classes in zip(texts, predicted_classes_per_sentence):
    # Из предложения создаем массив со словами
    sentence_words = sentence.split()
    # Проходимся по каждому слову и проверяем его класс
    for word, predicted_class in zip(sentence_words, predicted_classes):
        # Если предсказанный класс = "Навык", то добавляем в новый созданный массив
        if predicted_class == "Навык" and len(word) > 2:
            all_info[count][4].append(word)
    # Удаляем повторяющиеся навыки из массива 
    all_info[count][4] = list(set(all_info[count][4]))
    # Увеличиваем счетчик
    count += 1

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [306]:
# Импортурием из библиотеки scikit-learn модуль для работы с кластерами
from sklearn.cluster import KMeans
# Из бибилиотеки transformers загружаем предобученные модели на основе BERT
from transformers import BertTokenizer, BertModel
# Загружаем бибилиотеку для работы с нейросетями
import torch

# Загрузка токенизатора
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# Загрузка предобученной модели BERT
model = BertModel.from_pretrained('bert-base-uncased')

# Создаем массив для соединения всех навыком в одном месте
all_detected_skills = []
# Проходимся по всем данным
for i in range(len(all_info)):
    # Добавляем в ранее созданный массив навыки
    all_detected_skills += all_info[i][4]

# Максимальная длина последовательности
max_length = 32

# Векторизация текстов с учетом максимальной длины
encoded_texts = [tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length) for text in all_detected_skills]
input_ids = torch.cat([encoded_text['input_ids'] for encoded_text in encoded_texts], dim=0)
attention_masks = torch.cat([encoded_text['attention_mask'] for encoded_text in encoded_texts], dim=0)

# Получение эмбеддингов текстов с помощью BERT
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_masks)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

# Количество кластеров
num_clusters = 3  
# Добавляем модуль Kmeans для кластеризации
kmeans = KMeans(n_clusters=num_clusters)
# Фитим добавленный алгоритм
kmeans.fit(embeddings)

# Создание массивов для навыков каждого кластера
cluster_skills = [[] for _ in range(num_clusters)]

# Заполнение массивов навыками, соответствующими кластерам
for skill, cluster_label in zip(all_detected_skills, kmeans.labels_):
    cluster_skills[cluster_label].append(skill)

c:\Users\pupki\anaconda3\envs\nlp\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\pupki\anaconda3\envs\nlp\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


# 4. Анализ полученных данных путем NLP

  - #### 4.1 Определение наиболее востребованных и наименее востребованных навыков для выбранной группы вакансий

In [307]:
# Импортиурем из бибилиотеки модуль для счетчика
from collections import Counter

# Считаем количество вхождения всех навыков
skill_counts = Counter(all_detected_skills)

# Выводим наиболее востребованные навыки
print(f'Наиболее востребованные навыки: {skill_counts.most_common(10)}', end='\n\n')
# Выводим наименее востребованные навыки
print(f'Наиболее востребованные навыки: {skill_counts.most_common()[-10:]}')

Наиболее востребованные навыки: [('анализ', 9), ('аналитик', 9), ('данных', 9), ('sql', 9), ('знание', 8), ('разработка', 8), ('опыт', 8), ('бизнес', 7), ('данными', 7), ('python', 7)]

Наиболее востребованные навыки: [('talks', 1), ('tages', 1), ('кастомных', 1), ('конкурентный', 1), ('дротчеты', 1), ('собеседование', 1), ('meetups', 1), ('tagesteam', 1), ('инжиниринговый', 1), ('странахс', 1)]


   - ##### 4.2 Определение наиболее высокооплачеваемых навыков

In [308]:
# Создаем массив, куда сохраним наиболее высокооплачеваемые навыки
most_paid_skills = []
# Проходимся по всем данным
for i in range(len(all_info)):
    # Проверяем условие, если запрплата больше средней зарплаты всех вакансий, то добавляем в массив навыки
    if all_info[i][2] > 28906:
        most_paid_skills += all_info[i][4]
# Удаляем дубликаты навыков из массива
most_paid_skills = list(set(most_paid_skills))

# Выводим полученные результаты
print(f'Наиболее олпачеваемые навыки: {most_paid_skills}')

Наиболее олпачеваемые навыки: ['поставщик', 'powerbi', 'xml', 'знание', 'стартап', 'это', 'подарки', 'разработка', 'человекоориентированную', 'mysql', 'овертаймы', 'интегрировать', 'анализ', 'аналитик', 'вас', 'мвидео', 'qlikview', 'коммуникабельность', 'развивающийся', 'леруа', 'финансовый', 'быть', 'технониколь', 'аналитический', 'erp', 'неочевидный', 'tableu', 'усидчивость', 'бизнес', 'quotтехническийquot', 'аккредитованных', 'git', 'sheet', 'мдф', 'визуализация', 'как', 'tagesteam', 'фитнесс', 'собеседование', 'покупатель', 'данными', 'пожелание', 'производство', 'meetups', 'структурированный', 'очистка', 'data', 'лдсп', 'стрессоустойчивость', 'чтобы', 'дэшбордов', 'автоматизация', 'овертайм', 'зао', 'оптовый', 'полис', 'оформиться', 'аккредитация', 'аналитика', 'документооборот', 'квалификация', 'активно', '0800', 'прогнозирование', 'продажа', 'что', 'отличный', 'рост', 'science', 'администрирование', 'ему', 'представить', 'также', 'api', 'маркетплейсов', 'продавец', 'owner', 'sup

   - ##### 4.3 Определение региональной специфики востребованных навыков

In [309]:
# Импортиурем модуль для счетчика
from collections import Counter

# Создаем массив, куда сохраним наиболее востребованный навыки по регионам
most_needed_skills = {name: sum([i[4] for i in all_info if i[3]==name], []) for name in [i[3] for i in all_info]}
# Проходимся по полученному словарю
for region, skills in most_needed_skills.items():
    # Считаем количество вхождения навыка для региона
    skill_counts = Counter(skills)
    # Сохраняем навык с его счетчиком для региона
    most_needed_skills[region] = skill_counts.most_common(3)

# Выводим полученные результаты 
for key in most_needed_skills.keys():
    print(f'{key.capitalize()}: {most_needed_skills[key]}')

Новосибирская область: [('очистка', 1), ('tables', 1), ('xml', 1)]
Москва: [('аналитик', 5), ('данных', 5), ('python', 5)]
Нижегородская область: [('лдсп', 1), ('знание', 1), ('дэшбордов', 1)]
Свердловская область: [('знание', 2), ('разработка', 2), ('анализ', 2)]
Санкт-петербург: [('заработный', 1), ('себя', 1), ('кофеваркой', 1)]
Воронежская область: []
Брянская область: []
Республика татарстан: []
Московская область: []
Алтайский край: []


# 5. Передача данных в Excel

In [310]:
# Импортиурем библиотеку для работы с Excel на питоне
import openpyxl 
# Из бибилиотеки импортируем модуль счетчика
from collections import Counter

# Создаем массив для сохранения необходимых данных
data_for_excel = [[] for i in cluster_skills]
# Проходимся по всем кластерам
for i in range(len(cluster_skills)):
    # Считаем кол-во вхождений для каждого навыка в кластере
    skill_counter = Counter(cluster_skills[i])
    # Сохраняем в ранее созданный массив
    data_for_excel[i] = skill_counter

# Создание нового файла Excel
wb = openpyxl.Workbook()
# Получение активного листа
sheet = wb.active

# Заголовки столбцов
headers = ['Навык', 'Количество', 'Кластер']

# Запись заголовков в первую строку
for col, header in enumerate(headers, start=1):
    # Используем внутреннюю функцию бибилиотеки для записи названий стобцов
    sheet.cell(row=1, column=col, value=header)

# Стартовая строка для записи данных
row_index = 2

# Запись данных в таблицу для каждого кластера
for cluster_index, cluster in enumerate(data_for_excel, start=1):
    # Добавление заголовка для кластера
    sheet.cell(row=row_index, column=3, value=f'Кластер {cluster_index}')
    # Прибавляем к позиции строки +1
    row_index += 1
    # Запись данных из счетчика в таблицу
    for skill, count in cluster.items():
        sheet.cell(row=row_index, column=1, value=skill)
        sheet.cell(row=row_index, column=2, value=count)
        sheet.cell(row=row_index, column=3, value=f'Кластер {cluster_index}')  
        row_index += 1

# Сохранение файла Excel
wb.save('mainskills.xlsx')

____________________________________________________________________________________________________________________________________________________________________________________________________

###### Сохраняю данные про навыки для следующего модуля, для построения графа

In [24]:
# Ваш словарь данных с кортежами и их значениями
data = [i[4] for i in all_info]

# Открыть файл для записи
with open('dataModuleB.txt', 'w') as f:
    # Записать каждую пару кортеж и его значение в файл
    for value in data:
        f.write(f"{value},\n")